In [ ]:
from numpy import arange, nan, setdiff1d, sqrt
from numpy.random import choice, gamma, randint
from collections import defaultdict
from pandas import DataFrame

# AI Powered Business
### © 2020-2030 Horia Mărgărit

In [ ]:
sq2, sq3 = sqrt([2, 3]).astype('f4')

delivery_locations = arange(start=1, stop=71, dtype='i4').tolist()

drivers_vehicles = arange(start=1, stop=101, dtype='i4').tolist()

n_shifts_daily = randint(low=3, high=6, size=365, dtype='i4').tolist()

n_deliveries_daily = [randint(low=4, high=7, size=day, dtype='i4').tolist()
                      for day in n_shifts_daily]

In [ ]:
driver_ids = [choice(drivers_vehicles, size=day, replace=False).tolist()
              for day in n_shifts_daily]

driver_started = defaultdict(lambda: nan)
for i, drivers_daily in enumerate(driver_ids, start=1):
    for d_id in drivers_daily:
        if driver_started[d_id] is nan:
            driver_started[d_id] = i
        else:
            pass
del d_id, i, drivers_daily

location_ids = list()
for deliveries_per_shift in n_deliveries_daily:
    day = list()
    day_deliveries = choice(delivery_locations, size=sum(deliveries_per_shift), replace=False)
    for n_deliveries in deliveries_per_shift:
        day.append(choice(day_deliveries, size=n_deliveries, replace=False).tolist())
        day_deliveries = setdiff1d(day_deliveries, day[-1])
    location_ids.append(day)
del n_deliveries, deliveries_per_shift, day_deliveries, day

n_previous_deliveries = list()
cummulative_delivery_counts = defaultdict(lambda: 0)
for drivers_locations in zip(driver_ids, location_ids):
    day = list()
    for d_id, lids in zip(*drivers_locations):
        shift = list()
        for l_id in lids:
            shift.append(cummulative_delivery_counts[(d_id, l_id)])
            cummulative_delivery_counts[(d_id, l_id)] += 1
        day.append(shift)
    n_previous_deliveries.append(day)
del l_id, d_id, lids, shift, day, drivers_locations, cummulative_delivery_counts

performance_write_ups = [randint(low=0, high=4, size=day, dtype='i4').tolist()
                         for day in n_shifts_daily]

truck_maintenance_reqs = [randint(low=0, high=4, size=day, dtype='i4').tolist()
                         for day in n_shifts_daily]

vehicle_ids = [choice(drivers_vehicles, size=day, replace=False).tolist()
              for day in n_shifts_daily]

mu_driving_times = [[3. + 4.*pm[0] + 2.*pm[1]**2 for pm in zip(*day)]
                    for day in zip(performance_write_ups, truck_maintenance_reqs)]
consecutive_driving_time = [[gamma(shape=mu / sq3, scale=sq2, size=(n_deliveries_daily[i][j]+1,)).tolist()
                             for j, mu in enumerate(day)]
                            for i, day in enumerate(mu_driving_times)]
del mu_driving_times

In [ ]:
data = defaultdict(lambda: list())
for i, day in enumerate(zip(driver_ids,
                            vehicle_ids,
                            location_ids,
                            n_previous_deliveries,
                            performance_write_ups,
                            truck_maintenance_reqs,
                            consecutive_driving_time)):
    for j, shift in enumerate(zip(*day)):
        cum_prev_dels = 0
        prev_location = 'RS'
        for k, delivery in enumerate(zip(shift[2], shift[3], shift[6])):
            if k == len(shift[2])-1:
                curr_location = 'RS'
            elif k == len(shift[2]) // 2:
                data['day_index'].append(data['day_index'][-1])
                data['driver_id'].append(data['driver_id'][-1])
                data['vehicle_id'].append(data['vehicle_id'][-1])
                data['is_bad_weather'].append(data['is_bad_weather'][-1])
                data['is_bad_traffic'].append(data['is_bad_traffic'][-1])
                data['prev_location'].append(data['curr_location'][-1])
                data['curr_location'].append(data['curr_location'][-1])
                data['n_driver_days'].append(data['n_driver_days'][-1])
                data['n_performance_write_ups'].append(data['n_performance_write_ups'][-1])
                data['n_truck_maintenance_reqs'].append(data['n_truck_maintenance_reqs'][-1])
                data['n_location_penalty'].append(cum_prev_dels / sum(shift[3]) if sum(shift[3]) else 1)
                data['n_driving_minutes'].append(0.0)
                data['n_lunch_break_minutes'].append(60. - 45 * data['n_location_penalty'][-1])
                data['n_delivery_duration_minutes'].append(0.0)
                data['n_shift_duration_minutes'].append(sum(data['n_driving_minutes'][-k-1:]) + sum(data['n_lunch_break_minutes'][-k-1:]) + sum(data['n_delivery_duration_minutes'][-k-1:]))
            else:
                curr_location = '%.2d' % delivery[0]
            cum_prev_dels += delivery[1]
            data['day_index'].append(i+1)
            data['driver_id'].append(shift[0])
            data['vehicle_id'].append(shift[1])
            data['is_bad_weather'].append(1 if not randint(low=0, high=9, size=1, dtype='i4') else 0)
            data['is_bad_traffic'].append((1 if randint(low=0, high=5, size=1, dtype='i4') else 0)
                                          if data['is_bad_weather'][-1] else
                                          (1 if not randint(low=0, high=3, size=1, dtype='i4') else 0))
            data['prev_location'].append(prev_location)
            data['curr_location'].append(curr_location)
            data['n_driver_days'].append(i+1 - driver_started[shift[0]])
            data['n_performance_write_ups'].append(shift[4])
            data['n_truck_maintenance_reqs'].append(shift[5])
            data['n_location_penalty'].append(cum_prev_dels / sum(shift[3]) if sum(shift[3]) else 1)
            data['n_driving_minutes'].append(delivery[2] * (2 if data['is_bad_traffic'][-1] else 1))
            data['n_lunch_break_minutes'].append(0.0)
            data['n_delivery_duration_minutes'].append(0.0 if curr_location == 'RS' else 60. - 45 * data['n_location_penalty'][-1])
            data['n_shift_duration_minutes'].append(sum(data['n_driving_minutes'][-k-1:]) + sum(data['n_lunch_break_minutes'][-k-1:]) + sum(data['n_delivery_duration_minutes'][-k-1:]))
            prev_location = curr_location
del curr_location, prev_location, cum_prev_dels, k, delivery, j, shift, i, day

In [ ]:
df = DataFrame(data=data)
df.to_csv('../data/simulation.csv', index=False, header=True)